# Demonstrasjon av filformatet Apache Markdown med Python Pandas
Brønnøysundregistrene er en av mange aktører som tilbyr åpne data, både gjennom API-er og gjennom nedlasting av filer.
Du kan for eksempel laste ned hele eller deler av Enhetsregisteret som en JSON-fil eller Excel-fil. Det første formatet
er populært for de som skal bruke dataene ved hjelp av et programmeringsspråk, mens det siste er populært for alle andre.

Begge formatene har fordeler og ulemper. Fordelen med Excel-formatet er selvsagt at det gjør det veldig lett å bruke
dataene med regneark-program som Microsoft Excel. Og siden det er et så populært program, er det også mange andre
verktøy som støtter det formatet.

Stadig flere bruker Python og Pandas for å jobbe med data, både for å forberede dataene for analyse, og for selve analysen. 
Og mange bruker et "notebook"-verktøy, for å gjøre dette, slik som jeg har gjort her, gjennom en Jupyter Notebook.

En av grunnene til at det er lett å bruke Pandas er at det støtter Excel-formatet (og JSON-formatet), slik at det er veldig
lett å jobbe med data som finnes allerede i Excel, og å lagre resultatet av det du har gjort som Excel for å dele det med
andre.

Men Pandas støtter også formatet Apache Parquet. Nedenfor skal jeg demonstrere noen av fordelene med det formatet.

Men først, vi må få på plass de verktøyene vi trenger, og det er egentlig bare ett: Pandas

In [2]:
%pip install pandas

  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached numpy-1.21.4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install openpyxl

  Using cached openpyxl-3.0.9-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

## Laste ned og lese et passende utvalg av Enhetsregisteret som Excel
Vi starter med å bruke Pandas til å laste ned og lese inn en passende bit av Enhetsregisteret. 

In [3]:
enheter = pd.read_excel('https://data.brreg.no/enhetsregisteret/oppslag/enheter?fraRegistreringsdatoEnhetsregisteret=2021-11-01&tilRegistreringsdatoEnhetsregisteret=2021-11-21&sort=navn.norwegian,asc#', engine='openpyxl')

BadZipFile: File is not a zip file